# BIM relationship detection


Notebook for performing relationship detection using design files.

#### Input:
1. Design file structure extracted from navisworks NWD file.
2. IFC file generated from NWD file
3. graph predictions from GNN

#### Sections:
1. Relationship identification: Identify, extract and (visualize) 
aggregation and connectivity relationships.
2. Visalization: Deprecated
3. IFC to cloud: Create point clouds from each element in IFC file
4. Graph dataset: Create a graph dataset from relationship information
 
 (GNN training available in link_prediction.ipynb notebook)
5. Evaluate GNN: Evaluate predictions from GNN
6. Visalize predictions: Draw IFC element to visualize FP,TP,FNs
7. Analyze dataset and results: Repetition removal, element category analysis


In [ ]:
# setup

%load_ext autoreload
%autoreload 2

In [ ]:
# general
import json
import collections
import math
import uuid
import random
import pickle
import os
from itertools import islice
import numpy as np
from tqdm import tqdm

# ifc and pointcloud
import ifcopenshell
import open3d as o3d
import pymeshlab as ml
from compas.geometry import oriented_bounding_box_numpy
from scipy.spatial import distance
from ifcopenshell.util.selector import Selector

# graph 
import dgl
from dgl.data import DGLDataset
import torch

from src.structure import get_systems, get_branches
from src.visualisation import draw_relationship
from src.geometry import element_distance, 
from src.cloud import element_to_cloud
from src.graph import process_nodes, process_edges, IndustrialFacilityDataset

# vis
# from ipywidgets import interact
# from OCC.Core.Bnd import Bnd_Box, Bnd_OBB
# from OCC.Core.BRepBndLib import brepbndlib_AddOBB
# from OCC.Core.BRepPrimAPI import (BRepPrimAPI_MakeBox, 
# BRepPrimAPI_MakeSphere, BRepPrimAPI_MakeCylinder)
# from OCC.Core.gp import gp_Pnt, gp_XYZ, gp_Ax2, gp_Dir

# from utils.JupyterIFCRenderer import JupyterIFCRenderer

In [ ]:
# load design file structure

system_dict_file = "../EastDeckBox.nwd_aggregation.json"
system = 'East-DeckBox-Piping.rvm'

#m = ifcopenshell.open("data/231110AC-11-Smiley-West-04-07-2007.ifc")k

## Relationship Identification

###  aggrgegation relationships

In [ ]:
get_systems(system_dict_file)    

### Topological relationships

In [ ]:
branches = get_branches(system_dict_file)[system]

In [ ]:
# requisties for IFC file creation

create_guid = lambda: ifcopenshell.guid.compress(uuid.uuid1().hex)
m = ifcopenshell.open("../east_merged.ifc")
owner_history = m.by_type("IfcOwnerHistory")[0]
project = m.by_type("IfcProject")[0]
context = m.by_type("IfcGeometricRepresentationContext")[0]
floor = m.by_type("IfcBuildingStorey")[0]


In [ ]:
# draw links between connected elements, return connections
def visualize_branches(branches, ifc, floor=None, owner_history=None, context=None, draw=False, contiguous=True, dist_thresh=0.002):
    pipe_type = 'IFCPIPESEGMENT'
    fitting_type = 'IFCPIPEFITTING'

    pipe_selector = Selector()
    fitting_selector = Selector()
    pipes = pipe_selector.parse(ifc, '.' + pipe_type)
    fittings = fitting_selector.parse(ifc, '.' + fitting_type)
    fitting_names = [f.Name for f in fittings]
    pipe_names = [p.Name for p in pipes]
    print(pipes[0].Name)

    vis_dict = {}
    for k, val in branches.items():
        vis_elements = []
        connect = True
        for element in val:
            if element in pipe_names:
                vis_elements.append((element, pipe_type, connect))
                connect = True
            elif element in fitting_names:
                vis_elements.append((element, fitting_type, connect))
                connect = True
            else:
                connect = False
        vis_dict[k] = vis_elements

    error_count = 0
    count = 0
    rels = []
    selector = Selector()
    
    # enumerate through branches
    for k, val in tqdm(vis_dict.items()):
#         if count == 10:
#             break
        branch_size = len(val)
        for i, element in enumerate(val):
            #check if element is not the last element
            if (i+1) < branch_size:
                try:
                    
                    if val[i+1][2] or not contiguous:
                        rels.append([(element[0], element[1]), 
                                  (val[i+1][0], val[i+1][1])])
                        if draw:
                            element1 = selector.parse(
                                m, '.' + element[1] + '[Name *= "' + element[0] + '"]')[0]
                            element2 = selector.parse(
                                m, '.' + val[i+1][1] + '[Name *= "' + val[i+1][0] + '"]')[0]
                            draw_relationship(element[0], element1, 
                                  val[i+1][0], element2, ifc, floor, owner_history, context)
                
                    else:
                        element1 = selector.parse(
                            m, '.' + element[1] + '[Name *= "' + element[0] + '"]')[0]
                        element2 = selector.parse(
                            m, '.' + val[i+1][1] + '[Name *= "' + val[i+1][0] + '"]')[0]
                        
                        if element_distance(element1, element2, ifc) < dist_thresh:
                            rels.append([(element[0], element[1]), 
                                  (val[i+1][0], val[i+1][1])])
                            if draw:
                                draw_relationship(element[0], element1, 
                                  val[i+1][0], element2, ifc, floor, owner_history, context)
                except Exception as e:
                    #print (e)
                    error_count +=1
        count +=1

    print(error_count)
    return rels


In [ ]:
rels = visualize_branches(branches, m, floor, owner_history, context, True)
# rels = visualize_branches(branches, m)

In [ ]:
print(len(rels), rels[0])

In [ ]:
import pickle
with open('../top_rels_eastdeckbox_test.pkl', 'wb') as f:
    pickle.dump(rels, f)

In [ ]:
m.write('../east_vis_test.ifc')

## Visualization

In [ ]:
# viewer = JupyterIFCRenderer(m, size=(400,300))
# viewer.setAllTransparent()
# viewer

### aggregation relationships

In [ ]:
# picker = viewer.colorPicker()
# picker


In [ ]:
# out_dict = {'HVAC':['Rohrtypen:Kupfer - Hartgelötet:7718868', 'Rohrtypen:Kupfer - Hartgelötet:7718886', ],
#            'electrical':[ 'Rohrtypen:Kupfer - Hartgelötet:7718872', 'Rohrtypen:Kupfer - Hartgelötet:7718880']}

# # PAINT A SET OF ELEMENTS IN ONE COLOUR
# def systemSelect(system):
#     selector = Selector()
#     for e in out_dict[system]:
#         element = selector.parse(m, '.IfcProduct[Name *= "' + e + '"]')[0]
#         viewer.setColor(element, picker.value)
#     return system

# interact(systemSelect, system=['HVAC', 'electrical'])

Instances of building elements with represenations can be selected interactivly. Information such as the attributes `GUID`, `Name` etc. are displayed to the left of the 3D viewport.

In [ ]:
# # reset colours
# viewer.setDefaultColors()

### topological relationships



#### replace jupyter renderer

1. Compute the bounding box of ifc product directly from points
2. generate ifc elements to indicate relationships


In [ ]:
# test

# element_name = "TUBE 1 of BRANCH /AM-8120227-WD-MDA-01/B1"
# element_type = "IFCPIPESEGMENT"

# selector = Selector()
# element = selector.parse(
#     m, '.' + element_type + '[Name *= "' + element_name + '"]')[0]

# shape = element.Representation.Representations[0].Items[0]
# element_coords = np.array(shape.Coordinates.CoordList)
# #print(element_coords)
# bbox = oriented_bounding_box_numpy(element_coords)
# print(bbox)

In [ ]:
# test

# element1_name = "TUBE 1 of BRANCH /AM-8120227-WD-MDA-01/B1"
# element1_type = "IFCPIPESEGMENT"
# element2_name = "ELBOW 1 of BRANCH /AM-8120227-WD-MDA-01/B1"
# element2_type = "IFCPIPEFITTING"

# element3_name = "TUBE 2 of BRANCH /AM-8120227-WD-MDA-01/B1"
# element3_type = "IFCPIPESEGMENT"
# element4_name = "ELBOW 2 of BRANCH /AM-8120227-WD-MDA-01/B1"
# element4_type = "IFCPIPEFITTING"
# element5_name = "TUBE 3 of BRANCH /AM-8120227-WD-MDA-01/B1"
# element5_type = "IFCPIPESEGMENT"

# draw_relationship(element1_name, element1_type, 
#                   element2_name, element2_type, m)
# draw_relationship(element2_name, element2_type, 
#                   element3_name, element3_type, m)
# draw_relationship(element3_name, element3_type, 
#                   element4_name, element4_type, m)
# draw_relationship(element4_name, element4_type, 
#                   element5_name, element5_type, m)
#element1_center, element1_coords = get_element_deets()



# centerpoint =gp_Pnt(element1_center)
# ball = BRepPrimAPI_MakeSphere(centerpoint, 0.02).Shape()

In [ ]:
# element.Representation.Representations[0].Items[0].CoordIndex = element.Representation.Representations[0].Items[0].CoordIndex[:1]

## IFC to cloud

sample points from ifc model

In [ ]:
ifc = ifcopenshell.open("../east_merged.ifc")

In [ ]:
element_type = 'IFCPIPESEGMENT'
selector = Selector()
tees = selector.parse(ifc, '.' + element_type)
print(len(tees))

In [ ]:
for i, element in tqdm(enumerate(tees)):
    save_path = "../east_tee_clouds_test/" + str(i) + ".ply"
    cloud = element_to_cloud(element, save_path, 1000)

## Graph dataset

### get nodes & edges

In [ ]:
types = ['FLANGE', 'ELBOW', 'TEE', 'TUBE', 'BEND']
node_info = process_nodes(ifc, types)

In [ ]:
print(len(node_info), len(node_info[0]), len(node_info[1]))

with open('../nodes_eastdeckbox_test.pkl', 'wb') as f:
    pickle.dump(node_info, f)

In [ ]:
with open('../top_rels_eastdeckbox.pkl', 'rb') as f:
    rels = pickle.load(f)
with open('../nodes_eastdeckbox.pkl', 'rb') as f:
    node_info = pickle.load(f)
    nodes = node_info[0]
    
edges = process_edges(ifc, nodes, rels)

In [ ]:
with open('../edges_eastdeckbox_test.pkl', 'wb') as f:
    pickle.dump(edges, f)

In [ ]:
# sanity checks

# print(points.shape, labels.shape, centers.shape, lengths.shape, directions.shape)
# print(points[0][0], labels[0], centers[0], lengths[0], directions[0])
# print(5 in labels)

# edges_src = edges[:,0]
# edges_dst = edges[:,1]
# print(edges_src)
# print(np.max(edges_dst))

### create dataset

In [ ]:
dataset = IndustrialFacilityDataset()
graph = dataset[0]

print(graph)

## Evaluate GNN

In [ ]:
# load edges
with open('../eval/pos_edges_test.pkl', 'rb') as f:
            u, v = pickle.load(f)
with open('../eval/neg_edges_test.pkl', 'rb') as f:
            neg_u_full,neg_v_full = pickle.load(f)
print(len(neg_u_full), len(u))


In [ ]:
# load predicted scores
with open('../eval/pos_score_test.pkl', 'rb') as f:
            pos_score = pickle.load(f)
with open('../eval/neg_score_test.pkl', 'rb') as f:
            neg_score= pickle.load(f)


In [ ]:
print(len(neg_score), len(pos_score))

Calculate confusion matrix coefficiants

In [ ]:
threshold = 0.5
FNs, TPs, FPs, TNs = [], [], [], []

# TPs / FNs
for i, score in enumerate(pos_score):
    sig = 1/(1 + np.exp(-score))
    if sig > threshold:
        TPs.append([u[i].item(), v[i].item()])
    else:
        FNs.append([u[i].item(), v[i].item()])

# FPs
for i, score in tqdm(enumerate(neg_score)):
    sig = 1/(1 + np.exp(-score))
    if sig > threshold:
        FPs.append([neg_u_full[i], neg_v_full[i]])

print(len(TPs), len(FPs), len(FNs) )
TPs = np.array(TPs)
FPs = np.array(FPs)
FNs = np.array(FNs)

In [ ]:
print(1/(1 + np.exp(-neg_score[0])))

In [ ]:
with open('../eval/metrics_test.pkl', 'wb') as f:
    pickle.dump([TPs, FPs, FNs], f)

In [ ]:
with open('../eval/metrics_test.pkl', 'rb') as f:
    TPs, FPs, FNs = pickle.load(f)

Calculate raw metrics

In [ ]:
precision = len(TPs)/(len(TPs)+len(FPs))
recall = len(TPs)/(len(TPs)+len(FNs))
accuracy = (len(TPs)-len(FPs) +len(neg_score))/(len(neg_score)+len(pos_score))
print(precision, recall, accuracy)

Refinement based on element distances (eliminate predictions beyond a distance threshold)

In [ ]:
site = 'eastdeckbox'
data_path = '../'
node_file = "nodes_" + site + ".pkl"
with open(data_path + node_file, 'rb') as f:
    node_info = pickle.load(f)

In [ ]:
# check if positive predictions fall within distance threshold
def check_predictions(preds, point_info, dist_thresh=0.002):
    refined_preds = []
    
    for pair in tqdm(preds):
        dist = np.min(distance.cdist(
            point_info[pair[0]], point_info[pair[1]], 'sqeuclidean'))
        if (dist < dist_thresh):
            refined_preds.append(pair)
    return refined_preds

In [ ]:
refined_TPs = check_predictions(TPs, node_info[1])
refined_FPs = check_predictions(FPs, node_info[1])

print(len(refined_TPs), len(refined_FPs))

In [ ]:
with open('../eval/refined_test.pkl', 'wb') as f:
    pickle.dump([refined_TPs, refined_FPs], f)

In [ ]:
with open('../eval/refined.pkl', 'rb') as f:
    refined_TPs, refined_FPs = pickle.load(f)
print(len(refined_TPs))

Refined metrics

In [ ]:
precision = len(refined_TPs)/(len(refined_TPs)+len(refined_FPs))
recall = len(refined_TPs)/(len(refined_TPs)+len(FNs))
accuracy = (len(refined_TPs)-len(refined_FPs) +len(neg_score))/(len(neg_score)+len(pos_score))
print(precision, recall, accuracy)

ROC curve

In [ ]:
pos_score= pos_score[:int(len(pos_score)/10)]
neg_score= pos_score[:int(len(neg_score)/10)]

In [ ]:
len(neg_score)

In [ ]:
scores = np.concatenate([pos_score, neg_score])

In [ ]:
sig = 1/(1 + np.exp(-scores))

In [ ]:
from sklearn.metrics import RocCurveDisplay
import matplotlib.pyplot as plt


RocCurveDisplay.from_predictions(labels, scores)
plt.show()

## visualize predictions

In [ ]:
ifc = ifcopenshell.open("../east_merged.ifc")

In [ ]:
create_guid = lambda: ifcopenshell.guid.compress(uuid.uuid1().hex)
owner_history = ifc.by_type("IfcOwnerHistory")[0]
project = ifc.by_type("IfcProject")[0]
context = ifc.by_type("IfcGeometricRepresentationContext")[0]
floor = ifc.by_type("IfcBuildingStorey")[0]


In [ ]:
red = ifc.createIfcColourRgb('red', Red=0.9, Green=0.0, Blue=0.0)
green = ifc.createIfcColourRgb('green', Red=0.0, Green=0.9, Blue=0.0)
yellow = ifc.createIfcColourRgb('yellow', Red=0.9, Green=0.9, Blue=0.0)

In [ ]:
# visualize results on ifc file
def draw_predictions(preds, nodes, ifc, colour):
    for pair in tqdm(preds):
        element1 = ifc.by_id(nodes[pair[0]][4])
        element1_name = element1.Name
        element2 = ifc.by_id(nodes[pair[1]][4])
        element2_name = element2.Name
        
        draw_relationship(element1_name, element1, element2_name, 
                          element2, ifc, floor, owner_history, context, colour)

In [ ]:
draw_predictions(refined_TPs, node_info[0], ifc, green)

In [ ]:
draw_predictions(refined_FPs, node_info[0], ifc, yellow)

In [ ]:
draw_predictions(FNs, node_info[0], ifc, red)

In [ ]:
ifc.write('../eval/pred_vis_test.ifc')

## Additional refinements

### remove repetitions

In [ ]:
# as the graph is bidirected, a single edge has two predictions. 
# This function removes repetitions in a single set of predictions.
def remove_repetitions(preds):
    non_rep = []
    for i, pair in tqdm(enumerate(preds)):
        found = False
        for j, pair2 in enumerate(preds[i:]):
            if pair[0] == pair2[1] and pair[1] == pair2[0]:
                found = True
                #break
        if not found:
            non_rep.append(pair)
    
    return non_rep

In [ ]:
# same as above, except for removing repetitions across two sets of predictions
# ex. between true positives and false negatives
def compare_preds(preds1, preds2):
    for i in range(len(preds1)):
        preds1[i].sort()
    for i in range(len(preds2)):
        preds2[i].sort()
    non_rep = []
    
    for i, pair in enumerate(preds1):
        found = False
        for j, pair2 in enumerate(preds2):
            if pair[0] == pair2[0] and pair[1] == pair2[1]:
                found = True
                break
        if not found:
            non_rep.append(pair)
    
    return non_rep

In [ ]:
print(len(refined_TPs))
non_rep_TPs = remove_repetitions(refined_TPs)
print(len(non_rep_TPs))

In [ ]:
print(len(refined_FPs))
non_rep_FPs = remove_repetitions(refined_FPs)
print(len(non_rep_FPs))

In [ ]:
print(len(FNs))
non_rep_FNs = remove_repetitions(FNs)
print(len(non_rep_FNs))

In [ ]:
refined_FNs = compare_preds(non_rep_FNs, non_rep_TPs)
print(len(non_rep_FNs), len(refined_FNs))

Refined metrics after removing repetitions

In [ ]:
precision = len(non_rep_TPs)/(len(non_rep_TPs)+len(non_rep_FPs))
recall = len(non_rep_TPs)/(len(non_rep_TPs)+len(non_rep_FNs))
accuracy = (len(non_rep_TPs)-len(refined_FPs) +len(neg_score))/(len(neg_score)+len(pos_score))
print(precision, recall, accuracy)

In [ ]:
with open('../eval/east_non_rep_test.pkl', 'wb') as f:
    pickle.dump([non_rep_TPs, non_rep_FPs, non_rep_FNs], f)

Calculate metrics for each element category

### Analyse dataset and results

In [ ]:
# element type wise precision recall analysis
def sort_type(preds, nodes):
    bins = np.zeros([4,4])
    for p in preds:
        x=nodes[p[0]][0]
        y= nodes[p[1]][0]
        if x == 4:
          x = 3
        if y == 4:
          y = 3
        li = [x,y]
        li.sort()
        x,y = li[0],li[1]
        
        bins[x][y] += 1
    return bins

In [ ]:
tp_bins = sort_type(non_rep_TPs,node_info[0] )
fp_bins = sort_type(non_rep_FPs,node_info[0] )
fn_bins = sort_type(non_rep_FNs,node_info[0] )

In [ ]:
print(tp_bins)

In [ ]:
recall = tp_bins/(tp_bins+fn_bins)
precision = tp_bins/(tp_bins+fp_bins)

In [ ]:
print(precision)
print(recall)

In [ ]:
print("F1:", (2*precision*recall)/(precision+recall))

Calculate element types in dataset

In [ ]:
# analyse dataset

# load data
site = 'east'

def analyse_dataset(site):
  data_path = "/content/drive/MyDrive/graph/"
  edge_file = "edges_" + site + "deckbox.pkl"
  node_file = "nodes_" + site + "deckbox.pkl"
  with open(data_path + node_file, 'rb') as f:
      node_info = pickle.load(f)
  with open(data_path + edge_file, 'rb') as f:
      edges = pickle.load(f)

  # get element type counts
  labels = np.array([i[0] for i in node_info[0]])
  unique, counts = np.unique(labels, return_counts=True)
  print(dict(zip(unique, counts)))

  # get connection counts
  counts = np.zeros((5,5), dtype=int)

  for edge in edges:
    x = labels[edge[0]]
    y = labels[edge[1]]
    if x == 4:
      x = 3
    if y == 4:
      y = 3
    li = [x,y]
    li.sort()
    x,y = li[0],li[1]
    counts[x][y] += 1

  return(counts)

c_east = analyse_dataset('east')
c_west = analyse_dataset('west')
count = c_east + c_west
print(count)

<br><br><br><br>